# Generate data for clustering models - Mt Bolton

In [1]:
import numpy as np
import pyart
import os
#from sklearn.mixture import GaussianMixture
import pickle
#from netCDF4 import num2date, date2num
#import math
from multiprocessing import Pool
from multiprocessing import cpu_count
import pandas as pd


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
def extract_vol(myradar,field):
    gatefilter = give_gatefilter(myradar)
    ex_gates = gatefilter._gate_excluded
    #print(field)
    #print(np.shape(ex_gates))
    if field == 'ALT':
        if filt:
            valid_returns = myradar.gate_z['data'][:][~ex_gates].flatten()
        else:
            valid_returns =  myradar.gate_z['data'][:].flatten()
            
        #print(np.shape(valid_returns))
    
    else:
        if filt:
            valid_returns = myradar.fields[field]['data'][:][~ex_gates].flatten()
            
        else:
            valid_returns = myradar.fields[field]['data'][:].flatten()
                
        valid_returns = valid_returns.data
    
    return valid_returns

def give_gatefilter(myradar):

    try:
        myradar.check_field_exists('SNR')
    except:
        myradar.add_field('SNR',
                          pyart.retrieve.calculate_snr_from_reflectivity(myradar,
                                                                         refl_field='DBZH',
                                                                         snr_field=None,
                                                                         toa=25000.0))

    # Set gatefilters
    gatefilter = pyart.correct.GateFilter(myradar)
    if VRADH_inside is not None:
        gatefilter.exclude_inside('VRADH',VRADH_inside[0],VRADH_inside[1])
    if VRADH_outside is not None:
        gatefilter.exclude_outside('VRADH',VRADH_outside[0],VRADH_outside[1])
    if snr_cutoff is not None:
        gatefilter.exclude_below('SNR',snr_cutoff)
    if depseck_size is not None:
        gatefilter = pyart.correct.despeckle.despeckle_field(myradar,
                                                     'VRADH',
                                                     gatefilter=gatefilter,
                                                     size = depseck_size)
    return gatefilter

def extract_field_dict(vol_no): 
    try:
        myradar = pyart.aux_io.read_odim_h5('/'.join([dirr,all_files[vol_no]]), file_field_names=True)
    except:
        print('Skipped ' + all_files[vol_no] + ', error opening file')
        return
    
    if np.round(myradar.azimuth['data'][0]) != 185.0:
        print('Skipped ' + all_files[vol_no] + ', RHI scan is not 185 deg')
        return
    
    flattened_data = {}
    for field in fields_to_extract:
        flattened_data[field] = []
        flattened_data[field].extend(extract_vol(myradar,field))
            
    if retrieve_time:
        flattened_data['time']= np.full_like(flattened_data[0],myradar.time['data'][0])
        
    df = pd.DataFrame(flattened_data)
    
    #Do any data cleaning here. This could also be done using the gatefilter
    
    df = clean_df(df)
    
    return df

def handle_output(result):
    if result is None:
        return
    else:
        try:
            store = pd.HDFStore(outloc+eventname+'.h5')
            store.append(eventname, result, format='t',  data_columns=True)
            store.close()
            #print('Saved')
        except Exception as e:
            print('Saving hdf5 file failed with error: ')
            print(e)
            
def clean_df(df):
    
    #Do any data cleaning here. This could also be done using the gatefilter
    
    #Change order
    df = df[fields_to_extract]
    #Drop NaN
    df = df.dropna(axis=1, how='any')
    #Limit RhoHV to 1
    df['RHOHV'] = np.where(df['RHOHV'] >= 1, 1,df['RHOHV'])
    #Exclude values where ZDR is outside bounds RhoHV to 1
    df = df[(df['ZDR'] >= -7) & (df['ZDR'] <= 7)]
    #Exclude extremes of dBZ
    df = df[(df['DBZH'] >= -10) & (df['DBZH'] <= 40)]
    #Round Altitude to the nearest metre
    df = df = df.round({'ALT': 0})
    
    return df


In [3]:
test = False

fields_to_extract = ['DBZH','ZDR','RHOHV','WRADH','KDP','ALT']

filt = True

#Exclude values above this SNR value
snr_cutoff = 2
#Exclude values inside these values
VRADH_inside = None #[-0.5,0.5]
#Exclude values outside these values
VRADH_outside = None #[-10,10]
#Apply despeckle filter on VRADH with this minimum no of pixels
depseck_size = None #10

outloc = './training_data/'

RHI = True

retrieve_height = False
retrieve_time = False

In [4]:
if __name__ == '__main__':
    
    num_processes = cpu_count()

    eventname = 'Mt_Bolton_height_185'
    dirr = './raw_data/Mt Bolton/radar_hdf/'
    
    all_files = os.listdir(dirr)
    all_files.sort()
    
    if test:
        all_files = [all_files[187]]
    
    print('Begin processing ' + str(len(all_files)) + ' files.')
    pool = Pool(4)
    for i in range(len(all_files)):
        pool.apply_async(extract_field_dict, (i, ), callback=handle_output)
    pool.close()
    pool.join()
    

Begin processing 3530 files.
Skipped uq-xpol_rhi_20160223_033616.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033611.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033625.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033620.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033629.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033634.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033643.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033638.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033647.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033651.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033656.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033700.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033705.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033709.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_033714.h5, RHI scan is not 185 deg
Skipped uq-

Skipped uq-xpol_rhi_20160223_034556.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034552.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034601.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034605.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034610.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034614.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034619.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034623.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034628.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034632.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034637.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034641.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034645.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034650.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034654.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_034659.h5, 

Skipped uq-xpol_rhi_20160223_040112.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040116.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040125.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040120.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040134.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040129.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040143.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040138.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040152.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040147.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040200.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040156.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040205.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040209.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040214.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_040218.h5, 

Skipped uq-xpol_rhi_20160223_052039.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052043.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052050.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052046.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052053.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052057.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052100.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052104.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052107.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052111.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052114.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052118.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052121.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052125.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052128.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052132.h5, 

Skipped uq-xpol_rhi_20160223_052812.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052815.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052823.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052819.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052830.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052826.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052833.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052837.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052840.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052844.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052847.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052851.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052854.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052858.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052901.h5, RHI scan is not 185 deg
Skipped uq-xpol_rhi_20160223_052905.h5, 

...

Skipped uq-xpol_sppi_20160223_061346.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063024.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063028.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063033.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063038.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063043.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063048.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063058.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063053.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063102.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063110.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063115.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063119.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063124.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_063129.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_2016

Skipped uq-xpol_sppi_20160223_064143.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064148.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064152.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064157.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064201.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064205.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064210.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064218.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064214.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064223.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064227.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064231.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064236.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064240.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_064245.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_2016

Skipped uq-xpol_sppi_20160223_065058.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065102.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065106.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065111.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065115.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065119.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065128.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065124.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065133.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065137.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065141.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065146.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065150.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065154.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_065159.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_2016

Skipped uq-xpol_sppi_20160223_070129.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070134.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070138.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070143.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070147.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070152.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070156.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070201.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070205.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070210.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070214.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070219.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070223.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070228.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_070232.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_2016

Skipped uq-xpol_sppi_20160223_071101.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071106.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071110.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071115.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071119.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071124.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071128.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071216.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071222.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071228.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071234.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071240.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071247.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071253.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_20160223_071259.h5, RHI scan is not 185 deg
Skipped uq-xpol_sppi_2016

In [19]:
df = pd.read_hdf('./training_data/Mt_Bolton_height.h5')
df.head()

,DBZH,ZDR,RHOHV,WRADH,KDP,ALT
3,3.91,1.96,0.872158,0.59,0.0,433.0
4,-1.23,-3.57,0.996979,2.19,0.0,519.0
5,-0.01,0.03,0.698782,1.75,0.0,606.0
6,-0.35,-5.55,0.671499,2.30,0.0,692.0
7,-2.62,-3.01,0.754753,2.27,0.0,779.0


In [20]:
df.describe()

,DBZH,ZDR,RHOHV,WRADH,KDP,ALT
count,4.744444e+07,4.744444e+07,4.744444e+07,4.744444e+07,4.744444e+07,4.744444e+07
mean,1.065416e+01,5.031372e-01,4.809681e-01,1.973775e+00,3.961350e-04,1.726232e+03
std,7.574095e+00,1.804982e+00,3.091700e-01,7.207711e-01,1.505521e-02,1.542697e+03
min,-1.000000e+01,-7.000000e+00,6.103702e-05,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.060000e+00,-3.400000e-01,2.132023e-01,1.460000e+00,0.000000e+00,4.740000e+02
50%,1.009000e+01,4.400000e-01,3.838618e-01,2.150000e+00,0.000000e+00,1.237000e+03
75%,1.581000e+01,1.430000e+00,7.885067e-01,2.570000e+00,0.000000e+00,2.680000e+03
max,4.000000e+01,7.000000e+00,1.000000e+00,3.200000e+00,2.230000e+00,5.000000e+04
